In [ ]:
import json
import pandas as pd
import os

import ipyannotations.text

In [ ]:
if 'ner_labels.json' in os.listdir('../data/reddit_scraping/'):
    with open('../data/reddit_scraping/ner_labels.json') as json_file:
        ner_labels = json.load(json_file)
        ner_labels = pd.DataFrame.from_dict([{'post_id': x['post_id'], 'context': x['context']} for x in ner_labels])
else:
    print("Do Token Classification Labelling First!")

In [ ]:
if 'classification_labelsy.json' in os.listdir('../data/reddit_scraping/'):
    with open('../data/reddit_scraping/classification_labels.json') as json_file:
        classification_labels = pd.DataFrame.from_dict(json.load(json_file))
else:
    classification_labels = pd.DataFrame(columns = ['post_id'])

In [ ]:
posts_to_label = ner_labels[~ner_labels['post_id'].isin(classification_labels['post_id'])]

In [ ]:
posts = posts_to_label['context'].tolist()
gender_labels = []
widget = ipyannotations.text.ClassLabeller(options=['Male', 'Female', 'Non-Binary', 'Not Disclosed'], allow_freetext=False)

def store_annotations(gender_label):
    gender_labels.append(gender_label)
    try:
        widget.display(
            f"""
            {len(posts)} Posts left to annotate
            
            {posts.pop(0)}
            """
        )
    except IndexError:
        posts_to_label['gender_label'] = gender_labels
        widget.clear()

widget.on_submit(store_annotations)
widget.display(posts.pop(0))
widget

In [ ]:
posts = posts_to_label['context'].tolist()
subject_labels = []
widget = ipyannotations.text.ClassLabeller(options=['Self', 'Other'], allow_freetext=False)

def store_annotations(subject_label):
    subject_labels.append(subject_label)
    try:
        widget.display(
            f"""
            {len(posts)} Posts left to annotate
            
            {posts.pop(0)}
            """
        )
    except IndexError:
        posts_to_label['subject_label'] = subject_labels
        widget.clear()

widget.on_submit(store_annotations)
widget.display(posts.pop(0))
widget

In [ ]:
classification_labels = pd.concat([classification_labels, posts_to_label]).to_dict(orient = 'records')
with open('../data/reddit_scraping/classification_labels.json', 'w') as reddit_file:
    json.dump(classification_labels, reddit_file, indent=4)